In [1]:
%%capture --no-stderr
%pip install langchain langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 4.5 MB/s eta 0

In [7]:
from google.colab import userdata
openai_key = userdata.get('openai_key')

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_key)

In [9]:
!pip install langchainhub

In [10]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [12]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=openai_key))

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a complex task into smaller, manageable steps to simplify execution. This can be achieved through techniques like Chain of Thought (CoT) prompting, which encourages models to think step by step, or through methods like Tree of Thoughts that explore multiple reasoning possibilities. It can also involve human inputs or task-specific instructions to guide the decomposition.'

# 실습

**1. 3개의 블로그 포스팅 본문을 Load: WebBaseLoader 활용**

In [65]:
def load_documents(urls):
  loader = WebBaseLoader(
    web_paths=(urls),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
  )

  return loader.load()

**2. 불러온 본문을 Split (Chunking) : recursive text splitter 활용**

In [66]:
def split_documents(docs):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  return text_splitter.split_documents(docs)

# print(splits[0])

**3. Chunks 를 임베딩하여 Vector store 저장: openai, chroma 사용**

embedding model 은 "text-embedding-3-small" 사용

In [68]:
def embed(splits):
  return Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small", api_key=openai_key))

**4. User query = ‘agent memory’ 를 받아 관련된 chunks를 retrieve**

In [69]:
# user_query = "agent memory"

def docs_retriever(vectorstore, user_query):
  retriever = vectorstore.as_retriever()

  return retriever.invoke(user_query)

In [70]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = load_documents(urls)
splits = split_documents(docs)
vectorstore = embed(splits)
retrieved_docs = docs_retriever(vectorstore, "agent memory")

print(retrieved_docs)

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.\n\nEach element is an observation, an event directly provided by the agent.\n- Inter-agent communication can trigger new natural language statements.\n\n\nRetrieval model: surfaces the context to inform the agent’s behavior, according to relevance, recency and importance.\n\nRecency: recent events have higher scores\nImportance: distinguish mundane from core memories. Ask LM directly.\nRelevance: based on how related it is to the current situation / query.\n\n\nReflection mechanism: synthesizes memories into higher level inferences over time and guides the agent’s future behavior. They are higher-level summaries of past events (<- note that this is a bit different from self-reflection above)'), Document(metadata={'source': 'https://lilianweng.githu

**5. User query와 retrieved chunk 에 대해 relevance 가 있는지를 평가하는 시스템 프롬프트 작성: retrieval 퀄리티를 LLM 이 스스로 평가하도록 하고, 관련이 있으면 {‘relevance’: ‘yes’} 관련이 없으면 {‘relevance’: ‘no’} 라고 출력하도록 함. ( JsonOutputParser() 를 활용 )**

In [59]:
import json
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

system_prompt = "Evaluate the relevance between the User query and the retrieved chunk. If relevant, output {{\"relevance\": \"yes\"}}.; if not relevant, output {{\"relevance\": \"no\"}}. "
user_prompt = """
User query:
{user_query}

Retrieved chunk:
{retrieved_chunk}

Evaluation result:
"relevance": {{yes or no}}
"""

parser = JsonOutputParser()

template = ChatPromptTemplate([
    ("system", system_prompt),
    ("human", user_prompt)
]).partial(format_instructions=parser.get_format_instructions())

chain = template | llm | parser
response = chain.invoke({"user_query": user_query, "retrieved_chunk": docs})

print(response['relevance'])


yes


**6. 5 에서 모든 docs에 대해 ‘no’ 라면 디버깅 (Splitter, Chunk size, overlap, embedding model, vector store, retrieval 평가 시스템 프롬프트 등)**

**7. 5에서 ‘yes’ 라면 질문과 명확히 관련 없는 docs 나 질문 (예: ‘I like an apple’)에 대해서는 ‘no’ 라고 나오는지 테스트 프롬프트 및 평가 코드 작성. 이 때는 관련 없다는 답변 작성**

In [60]:
user_query = "I like an apple"
response = chain.invoke({"user_query": user_query, "retrieved_chunk": docs})

print(response['relevance'])

no


**8. ‘yes’ 이고 7의 평가에서도 문제가 없다면, 4의 retrieved chunk 를 가지고 답변 작성 prompt | llm | parser 코드 작성**

In [63]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Task Decomposition is the process of breaking down a complex task into smaller, manageable steps or subgoals. This can be achieved through various methods, such as prompting a language model, using task-specific instructions, or incorporating human inputs. The approach facilitates clearer planning and execution by allowing for a structured pathway to tackle the original task.

**9. 생성된 답안에 Hallucination 이 있는지 평가하는 시스템 프롬프트 작성. LLM이 스스로 평가하도록 하고, hallucination 이 있으면 {‘hallucination’: ‘yes’} 없으면 {‘hallucination’: ‘no’} 라고 출력하도록 함**

**10. 9 에서 ‘yes’ 면 8 로 돌아가서 다시 생성, ‘no’ 면 답변 생성하고 유저에게 답변 생성에 사용된 출처와 함께 출력 (최대 2번까지 다시 생성)**

11. https://applied-llms.org/ 링크는 llm 을 이용해 1년간 개발해 본 팀이 배운 것들을 정리한 아티클 입니다.

저자의 생각을 묻는 질문들에 답하기 위해서

1) Chunking & embedding & storing

2) Load

3) Retrieval

4) Generation

으로 구성된 RAG 코드를 작성하고 아래 예시 질문에 대한 답을 해보세요.

예시)
RAG 에 대한 저자의 생각은 무엇인가?
RAG 와 fine tuning 에 대해 저자는 어떻게 비교하고 있나?
저자가 가장 많은 부분을 할당해 설명하는 개념은 무엇인가?

11 실습을 마치지 못했더라도, https://applied-llms.org/ 아티클은 꼼꼼히 정독해보시기를 추천 드립니다.